In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv(r"C:\Users\ashuv\Desktop\csv\Churn_Modelling.csv")

In [3]:
data.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [4]:
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [6]:
#X = data.iloc[:,3:13].values

In [5]:
X = data.drop(columns = ["Exited",'RowNumber','CustomerId','Surname'])

In [7]:
X= pd.get_dummies(X, columns=['Geography','Gender'])

In [8]:
y = data.iloc[:,13].values

In [9]:
y = data["Exited"]

In [13]:
X.shape

(10000, 13)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state =4)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X= sc.fit_transform(X)
#X_test = sc.transform(X_test)

In [33]:
from keras.utils import np_utils
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

Y_train = np_utils.to_categorical(y_train, 26)
Y_test = np_utils.to_categorical(y_test, 26)

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [30]:
ann = Sequential()

# Adding the input layer and the first hidden layer
ann.add(Dense(units=13, activation='relu'))

# Adding the second hidden layer
ann.add(Dense(units=6, activation='relu'))

# Adding the output layer
ann.add(Dense(units=1, activation='softplus'))
# Part 3 - Training the ANN

# Compiling the ANN
import tensorflow as tf
opt = tf.keras.optimizers.Adam()
#opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)

ann.compile(loss="binary_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

In [31]:
ann.fit(X_train, y_train, batch_size = 32, validation_data = (X_test,y_test), epochs = 50)

Epoch 1/50
250/250 [==============================] - 1s 1ms/step - loss: 0.6474 - accuracy: 0.6551 - val_loss: 0.4802 - val_accuracy: 0.8045
Epoch 2/50
250/250 [==============================] - 0s 812us/step - loss: 0.4640 - accuracy: 0.8035 - val_loss: 0.4354 - val_accuracy: 0.8185
Epoch 3/50
250/250 [==============================] - 0s 798us/step - loss: 0.4386 - accuracy: 0.8112 - val_loss: 0.4205 - val_accuracy: 0.8255
Epoch 4/50
250/250 [==============================] - 0s 824us/step - loss: 0.4260 - accuracy: 0.8176 - val_loss: 0.4133 - val_accuracy: 0.8240
Epoch 5/50
250/250 [==============================] - 0s 850us/step - loss: 0.4161 - accuracy: 0.8216 - val_loss: 0.4032 - val_accuracy: 0.8370
Epoch 6/50
250/250 [==============================] - 0s 800us/step - loss: 0.4068 - accuracy: 0.8301 - val_loss: 0.3951 - val_accuracy: 0.8395
Epoch 7/50
250/250 [==============================] - 0s 889us/step - loss: 0.3984 - accuracy: 0.8341 - val_loss: 0.3908 - val_accuracy: 0

In [32]:
y_pred = ann.predict(X_test)

In [33]:
y_pred = (y_pred > 0.5)

In [34]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [False]])

In [35]:
ann.evaluate(X_test, y_test)

63/63 [==============================] - 0s 581us/step - loss: 0.3421 - accuracy: 0.8720


[0.34211182594299316, 0.871999979019165]

In [36]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [37]:
cm

array([[1542,   62],
       [ 194,  202]], dtype=int64)

In [38]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=1000)

In [39]:
clf.score(X_test,y_test)

0.874

In [10]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)
X = pca.fit_transform(X)


In [12]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=1000)

In [13]:
X.shape

(60000, 300)

In [14]:
clf.score(X_test,y_test)

0.9428333333333333